### S50 - ACTUALIZAR DATOS DEL HEADER ya Existentes en CONNEXA 

Parte de los forecast executión que están en estado 50 (Ya publicados OK), Genera los datos acumulados del registro excec y sube los archivos a connexa

In [ ]:
# Importar librerías estándar
import os
import shutil
import time
from datetime import datetime
from random import randint

# Importar librerías de terceros
import pandas as pd
import numpy as np
from dotenv import dotenv_values

# Importar funciones necesarias del módulo `funciones_forecast`
from funciones_forecast import (
    Open_Conn_Postgres,
    Open_Connection,
    Close_Connection,
    get_execution_by_status,
    Open_Postgres_retry,
    mover_archivos_procesados,
    actualizar_site_ids,
    insertar_graficos_forecast,
    get_precios,
    get_execution_execute_by_status,
    update_execution,
    update_execution_execute,
    create_execution_execute_result,
    generar_mini_grafico,
    generar_grafico_base64
)

# Importar librerías adicionales necesarias
import ace_tools_open as tools

# Cargar configuraciones desde archivo `.env`
secrets = dotenv_values(".env")
folder = secrets["FOLDER_DATOS"]



In [ ]:
fes = get_execution_execute_by_status(50)
tools.display_dataframe_to_user(name="Contenido de Archivos Markdown", dataframe=fes)

# EJECUTABLE PYTHON

In [ ]:
def obtener_datos_stock(id_proveedor, etiqueta):
    secrets = dotenv_values(".env")   # Connection String from .env
    folder = secrets["FOLDER_DATOS"]
    
    #  Intento recuperar datos cacheados
    try:         
        print(f"-> Generando datos para ID: {id_proveedor}, Label: {etiqueta}")
        # Configuración de conexión
        conn = Open_Connection()
        
        # ----------------------------------------------------------------
        # FILTRA solo PRODUCTOS HABILITADOS y Traer datos de STOCK y PENDIENTES desde PRODUCCIÓN
        # ----------------------------------------------------------------
        query = f"""              
        SELECT A.[C_PROVEEDOR_PRIMARIO] as Codigo_Proveedor
            ,S.[C_ARTICULO] as Codigo_Articulo
            ,S.[C_SUCU_EMPR] as Codigo_Sucursal
            ,S.[I_PRECIO_VTA] as Precio_Venta
            ,S.[I_COSTO_ESTADISTICO] as Precio_Costo
            ,S.[Q_FACTOR_VTA_SUCU] as Factor_Venta
            ,ST.Q_UNID_ARTICULO + ST.Q_PESO_ARTICULO AS Stock_Unidades-- Stock Cierre Dia Anterior
            ,(R.[Q_VENTA_30_DIAS] + R.[Q_VENTA_15_DIAS]) * S.[Q_FACTOR_VTA_SUCU] AS Venta_Unidades_30_Dias -- OJO convertida desde BULTOS DIARCO
                    
            ,(ST.Q_UNID_ARTICULO + ST.Q_PESO_ARTICULO)* S.[I_COSTO_ESTADISTICO] AS Stock_Valorizado-- Stock Cierre Dia Anterior
            ,(R.[Q_VENTA_30_DIAS] + R.[Q_VENTA_15_DIAS]) * S.[Q_FACTOR_VTA_SUCU] * S.[I_COSTO_ESTADISTICO] AS Venta_Valorizada

            ,ROUND(((ST.Q_UNID_ARTICULO + ST.Q_PESO_ARTICULO)* S.[I_COSTO_ESTADISTICO]) / 	
                ((R.[Q_VENTA_30_DIAS] + R.[Q_VENTA_15_DIAS]+0.0001) * S.[Q_FACTOR_VTA_SUCU] * S.[I_COSTO_ESTADISTICO] ),0) * 30
                AS Dias_Stock
                    
            ,S.[F_ULTIMA_VTA]
            ,S.[Q_VTA_ULTIMOS_15DIAS] * S.[Q_FACTOR_VTA_SUCU] AS VENTA_UNIDADES_1Q -- OJO esto está en BULTOS DIARCO
            ,S.[Q_VTA_ULTIMOS_30DIAS] * S.[Q_FACTOR_VTA_SUCU] AS VENTA_UNIDADES_2Q -- OJO esto está en BULTOS DIARCO
                
        FROM [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
        INNER JOIN [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
            ON A.[C_ARTICULO] = S.[C_ARTICULO]
        LEFT JOIN [DIARCOP001].[DiarcoP].[dbo].[T060_STOCK] ST
            ON ST.C_ARTICULO = S.[C_ARTICULO] 
            AND ST.C_SUCU_EMPR = S.[C_SUCU_EMPR]
        LEFT JOIN [DIARCOP001].[DiarcoP].[dbo].[T710_ESTADIS_REPOSICION] R
            ON R.[C_ARTICULO] = S.[C_ARTICULO]
            AND R.[C_SUCU_EMPR] = S.[C_SUCU_EMPR]

        WHERE S.[M_HABILITADO_SUCU] = 'S' -- Permitido Reponer
            AND A.M_BAJA = 'N'  -- Activo en Maestro Artículos
            AND A.[C_PROVEEDOR_PRIMARIO] = {id_proveedor} -- Solo del Proveedor
                        
        ORDER BY S.[C_ARTICULO],S.[C_SUCU_EMPR];
        """
        # Ejecutar la consulta SQL
        df_stock = pd.read_sql(query, conn)
        file_path = f'{folder}/{etiqueta}_Stock.csv'
        df_stock['Codigo_Proveedor']= df_stock['Codigo_Proveedor'].astype(int)
        df_stock['Codigo_Articulo']= df_stock['Codigo_Articulo'].astype(int)
        df_stock['Codigo_Sucursal']= df_stock['Codigo_Sucursal'].astype(int)
        df_stock.fillna(0, inplace= True)
        # df_stock.to_csv(file_path, index=False, encoding='utf-8')        
        print(f"---> Datos de STOCK guardados: {file_path}")
        return df_stock
    except Exception as e:
        print(f"Error en get_execution: {e}")
        return None
    finally:
        Close_Connection(conn)


def obtener_demora_oc(id_proveedor, etiqueta):
    secrets = dotenv_values(".env")   # Connection String from .env
    folder = secrets["FOLDER_DATOS"]
    
    #  Intento recuperar datos cacheados
    try:         
        print(f"-> Generando datos para ID: {id_proveedor}, Label: {etiqueta}")
        # Configuración de conexión
        conn = Open_Connection()
        
        # ----------------------------------------------------------------
        # FILTRA solo PRODUCTOS HABILITADOS y Traer datos de STOCK y PENDIENTES desde PRODUCCIÓN
        # ----------------------------------------------------------------
        query = f"""              
        SELECT  [C_OC]
            ,[U_PREFIJO_OC]
            ,[U_SUFIJO_OC]      
            ,[U_DIAS_LIMITE_ENTREGA]
            , DATEADD(DAY, [U_DIAS_LIMITE_ENTREGA], [F_ENTREGA]) as FECHA_LIMITE
            , DATEDIFF (DAY, DATEADD(DAY, [U_DIAS_LIMITE_ENTREGA], [F_ENTREGA]), GETDATE()) as Demora
            ,[C_PROVEEDOR] as Codigo_Proveedor
            ,[C_SUCU_COMPRA] as Codigo_Sucursal
            ,[C_SUCU_DESTINO]
            ,[C_SUCU_DESTINO_ALT]
            ,[C_SITUAC]
            ,[F_SITUAC]
            ,[F_ALTA_SIST]
            ,[F_EMISION]
            ,[F_ENTREGA]    
            ,[C_USUARIO_OPERADOR]    
            
        FROM [DIARCOP001].[DiarcoP].[dbo].[T080_OC_CABE]  
        WHERE [C_SITUAC] = 1
        AND C_PROVEEDOR = {id_proveedor} 
        AND DATEADD(DAY, [U_DIAS_LIMITE_ENTREGA], [F_ENTREGA]) < GETDATE();
        """
        # Ejecutar la consulta SQL
        df_demoras = pd.read_sql(query, conn)
        df_demoras['Codigo_Proveedor']= df_demoras['Codigo_Proveedor'].astype(int)
        df_demoras['Codigo_Sucursal']= df_demoras['Codigo_Sucursal'].astype(int)
        df_demoras['Demora']= df_demoras['Demora'].astype(int)
        df_demoras.fillna(0, inplace= True)         
        print(f"---> Datos de OC DEMORADAS Recuperados: {etiqueta}")
        return df_demoras
    except Exception as e:
        print(f"Error en get_execution: {e}")
        return None
    finally:
        Close_Connection(conn)

In [ ]:
 # Leer Dataframe de FORECAST EXECUTION  de Estado 50 y Actualizar HEADER
fes = get_execution_execute_by_status(50)

* 36	- 1 - 	Estaod OC Pendiente         -                        	Pendiente 
* 36	- 2 -	Estado OC Cerrada           -                       	Cumplida  
* 36	- 3 -	Estado OC Anulada           -                       	Anulada   

#### S50 ACTUALIZAR SOLO CABECERAS

In [ ]:
# elegido = '140_UNILEVER_ALGO_05'
# for index, row in fes[fes["name"] == elegido].iterrows():
    
for index, row in fes[fes["fee_status_id"] == 50].iterrows():
    algoritmo = row["name"]
    name = algoritmo.split('_ALGO')[0]
    execution_id = row["forecast_execution_id"]
    id_proveedor = row["ext_supplier_code"]
    forecast_execution_execute_id = row["forecast_execution_execute_id"]
    supplier_id = row["supplier_id"]
    
    folderP = folder + '/procesado'

    print(f"Algoritmo: {algoritmo}  - Name: {name} exce_id: {forecast_execution_execute_id} id: Proveedor {id_proveedor}")
    print(f"supplier-id: {supplier_id} ----------------------------------------------------")

    try:
        df_stock = obtener_datos_stock(id_proveedor= id_proveedor, etiqueta= algoritmo )
        total_stock_valorizado = float(round(df_stock['Stock_Valorizado'].sum() / 1000000, 2))
        total_venta_valorizada = float(round(df_stock['Venta_Valorizada'].sum() / 1000000, 2))
        days= int( total_stock_valorizado / total_venta_valorizada * 30 )
           
        # Definición de condiciones
        # Condiciones
        if days > 30:
            semaforo= 'green'
        elif 10 < days <= 30:
            semaforo ='yellow'
        elif days <= 10:
            semaforo ='red'
        else:
            semaforo = 'white' # Valor predeterminado

        df_demora = obtener_demora_oc(id_proveedor= id_proveedor, etiqueta= algoritmo )
        if df_demora.empty:  # Verifica si el DataFrame está vacío
            maximo_atraso_oc = 0
        else:
            maximo_atraso_oc = int(round(df_demora['Demora'].max()))

        # Actualizar en base de datos            
        update_execution_execute(
            forecast_execution_execute_id,
            # supply_forecast_execution_status_id=50,
            # monthly_sales_in_millions=total_venta,
            # monthly_purchases_in_millions=total_costo,
            # monthly_net_margin_in_millions=total_margen,
            # graphic=mini_grafico,
            # total_products=total_productos,
            # total_units=total_unidades,
            # otif = randint(70, 100),  # Simulación de OTIF entre 70 y 100
            sotck_days = days, # Simulación de stock_days entre 10 y 76              
            sotck_days_colors = semaforo, # Simulación de semaforo
            maximum_backorder_days = maximo_atraso_oc # Simulación de oc_delay entre 0 y 45
            
        )
        
        print(f"✅ Stock Actualizado para {execution_id}")
        
        # # ✅ Morver Archivo a carpeta de Procesado ....
        # mover_archivos_procesados(algoritmo, folder)
        # print(f"✅ Archivo movido a Procesado: {algoritmo}")

    except Exception as e:
        import traceback
        traceback.print_exc()

In [ ]:
print(df_stock.columns)

## PARCHES y ARREGLO DE PROBLEMAS

In [ ]:
    # ARREGLAR PROBLEMAS
df_sin_duplicados = df_forecast_ext.drop_duplicates(subset=['Codigo_Articulo', 'Sucursal'], keep='first')
file_path = f"{folder}/{algoritmo}_Pronostico_Extendido_Con_Graficos.csv"
df_sin_duplicados.to_csv(file_path, index=False)

In [ ]:
import matplotlib.pyplot as plt
import base64
from io import BytesIO
from IPython.display import display, Image

# Función para decodificar y mostrar una imagen en Jupyter Notebook
    
img_data = base64.b64decode(mini_grafico)
buffer = BytesIO(img_data)

# Mostrar la imagen en Jupyter Notebook
display(Image(buffer.getvalue()))


In [ ]:
# Actualizar en base de datos            
update_execution_execute(
    forecast_execution_execute_id,
    supply_forecast_execution_status_id=50,
    monthly_sales_in_millions=total_venta,
    monthly_purchases_in_millions=total_costo,
    monthly_net_margin_in_millions=total_margen,
    graphic=mini_grafico,
    total_products=total_productos,
    total_units=total_unidades,
    otif = randint(70, 100),  # Simulación de OTIF entre 70 y 100
    sotck_days = randint(10,76), # Simulación de stock_days entre 10 y 76
    sotck_days_colors ='green', # Simulación de semaforo
    maximum_backorder_days = randint(0,45) # Simulación de oc_delay entre 0 y 45
    
)